# Extreact reactions from mol entries

In [1]:
from pymatgen.reaction_network.reaction import bucket_mol_entries, unbucket_mol_entries
from pymatgen.reaction_network.reaction import IntramolSingleBondChangeReaction, IntermolecularReaction
from monty.serialization import dumpfn, loadfn

### Load molecule entries 

In [2]:
num_entries=2000
fname = f'/Users/mjwen/Documents/Dataset/rxnrep/mol_entries_n{num_entries}.json'

mol_entries = loadfn(fname)

print('number of mol entries loaded:', len(mol_entries))

number of mol entries loaded: 862


### Intra-molecular bond breaking reactions, i.e. A->B

In [3]:
buckets = bucket_mol_entries(mol_entries)
intra_mol_reactions, _ = IntramolSingleBondChangeReaction.generate(buckets)
len(intra_mol_reactions)

/Users/mjwen/Applications/pymatgen/pymatgen/reaction_network/reaction.py:2775: FutureWarning: Nbonds is deprecated
`Nbonds` is replaced by `num_bonds`. This will be removed shortly.
  v = getattr(m, k)
/Users/mjwen/Applications/pymatgen/pymatgen/reaction_network/reaction.py:689: FutureWarning: edges is deprecated
`edges` is replaced by `bonds`. This will be removed shortly.
  for edge in entry1.edges:


71

### Inter-molecular bond breaking reactions, i.e. A->B+C

In [4]:
buckets = bucket_mol_entries(mol_entries)
inter_mol_reactions, _ = IntermolecularReaction.generate(buckets)
len(inter_mol_reactions)

/Users/mjwen/Applications/pymatgen/pymatgen/reaction_network/reaction.py:975: FutureWarning: edges is deprecated
`edges` is replaced by `bonds`. This will be removed shortly.
  for edge in entry.edges:


36

### write reactions to file (pymatgen reactions)

In [5]:
reactions = intra_mol_reactions + inter_mol_reactions

fname = f'/Users/mjwen/Documents/Dataset/rxnrep/reactions_n{num_entries}.json'
dumpfn(reactions, fname)